<a href="https://colab.research.google.com/github/juelha/IANNWTF/blob/Sabine_branch/homework03_Sabine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

tfds.load('genomics_ood')

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.



Dl Completed...:   0%|          | 0/12 [00:00<?, ? file/s]


Dataset genomics_ood downloaded and prepared to /root/tensorflow_datasets/genomics_ood/0.0.1. Subsequent calls will reuse this data.


{'test': <PrefetchDataset shapes: {domain: (), label: (), seq: (), seq_info: ()}, types: {domain: tf.string, label: tf.int64, seq: tf.string, seq_info: tf.string}>,
 'test_ood': <PrefetchDataset shapes: {domain: (), label: (), seq: (), seq_info: ()}, types: {domain: tf.string, label: tf.int64, seq: tf.string, seq_info: tf.string}>,
 'train': <PrefetchDataset shapes: {domain: (), label: (), seq: (), seq_info: ()}, types: {domain: tf.string, label: tf.int64, seq: tf.string, seq_info: tf.string}>,
 'validation': <PrefetchDataset shapes: {domain: (), label: (), seq: (), seq_info: ()}, types: {domain: tf.string, label: tf.int64, seq: tf.string, seq_info: tf.string}>,
 'validation_ood': <PrefetchDataset shapes: {domain: (), label: (), seq: (), seq_info: ()}, types: {domain: tf.string, label: tf.int64, seq: tf.string, seq_info: tf.string}>}